# Проект для "Викишоп" с BERT

# ВВЕДЕНИЕ

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Требуется обучить модель классифицировать комментарии на позитивные и негативные. Исходные данные содержат разметку  токсичности правок.
Предполагается, что построенная модель будет иметь метрику качества F1 не меньше 0.75.

# Загрузка данных

Установка библиотек.

In [ ]:
!pip install imblearn -q

In [ ]:
!pip install wordcloud  -q

In [ ]:
!pip install transformers_interpret -q

In [ ]:
!pip install numpy==1.26.4

In [ ]:
import os
import time
import torch
import re

import pandas as pd
import nltk

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from tqdm import notebook
import copy

from nltk.corpus import stopwords as nltk_stopwords
from transformers import AutoModel, AutoTokenizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import f_classif, mutual_info_classif,SelectKBest
from sklearn.inspection import permutation_importance

from sklearn.model_selection import train_test_split

from sklearn.metrics import (
     accuracy_score,
     confusion_matrix,
     f1_score,
     ConfusionMatrixDisplay
)

from wordcloud import WordCloud
import transformers

import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


from sklearn.pipeline import Pipeline

RANDOM_STATE = 42
TEST_SIZE = 0.5

Загрузка данных.

In [ ]:
pth1 = 'toxic_comments.csv'
pth2 = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'

try:
    data = pd.read_csv(pth1)
except:
    data = pd.read_csv(pth2)

Просмотр загруженного датафрейма.

In [ ]:
display(data.head(10))

Общая информация о датафрейме, проверка на дубликаты и пропуски.

In [ ]:
data.info()

In [ ]:
data.duplicated().sum()

In [ ]:
data.isna().sum()

Посмотрим, сколько всего токсичных комментариев в выборке.

In [ ]:
data['toxic'].sum()/data['toxic'].count()

Около 10%, значит у нас будет сильный дисбаланс классов, который надо будет учесть при построении модели классификации.

 Сначала решим задачу классификации твитов, применив векторные представления на базе BERT. Построить векторы текстов нам поможет предобученная на токсичных  текстах модель toxic-bert.

# BERT

Таблица содержит почти 160 тысяч строк, это очень много для моделей обработки текста. Ограничимся случайной выборкой из 800 строк, посмотрим, как на них отработает модель.

In [ ]:
data_tox = data[data['toxic']==1]

In [ ]:
data_copy = data.copy(deep=True)#создадим копию исходного датафрейма

In [ ]:
data = data.sample(800).reset_index(drop=True)

Создаем список слов из корпуса токсичных комментариев и список слов из нейтральных.

In [ ]:
data_good = data[data['toxic']==0]

corpus_tox = list(data_tox['text'])
corpus_good = list(data_good['text'])

tox = ' '.join(corpus_tox)
good = ' '.join(corpus_good)

Построим облако слов отдельно для токсичных и для нейтральных комментариев для более наглядного представления.

Облако слов для токсичных комментариев (по всей выборке).

In [ ]:
start = time.perf_counter()

wordCloud = WordCloud(width = 10000, height = 10000, random_state=1, background_color='black', colormap='Set2', collocations=False).\
             generate(tox)
plt.figure(figsize=(10,10))
plt.imshow(wordCloud)

end=time.perf_counter()
elapsed =end - start
print(f"Затрачено времени: {elapsed} сек")

Облако слов для нейтральных комментариев (по сэмплированной выборке).

In [ ]:
start=time.perf_counter()

wordCloud = WordCloud(width = 10000, height = 10000, random_state=1, background_color='black', colormap='Set2', collocations=False).\
             generate(good)
plt.figure(figsize=(10,10))
plt.imshow(wordCloud)

end=time.perf_counter()
elapsed =end - start
print(f"Затрачено времени: {elapsed} сек")

Загружаем список стоп-слов английского языка.

In [ ]:
nltk.download('stopwords')

stopwords = set(nltk_stopwords.words('english'))

Создаем корпус текстов.

In [ ]:
corpus = list(data['text'])

Функция для очистки текста от ненужных символов.

In [ ]:
def clear_text(text):
    t=re.sub(r'[^a-zA-Z]', ' ', text)
    t=t.split()
    return ' '.join(t)

Теперь применяем эту функцию к каждому предложению из корпуса.

In [ ]:
a = []
for i in corpus:
    a.append(clear_text(i))

Переписываем столбец data['text'] и проверяем корректность нашей обработки.

In [ ]:
data['text'] = a

In [ ]:
display(data.head(10))

Таким образом, наш текст очищен от лишних символов, теперь к нему можно применять модель для создания признаков для задачи классификации и прогноза токсичности отзывов.

### Подготовка признаков для задачи классификации


Преобразуем исходные тексты в векторы, которые и будут признаками в задаче классификации. Для этого используем предобученную модель "unitary/toxic-bert".

Инициализируем модель и токенизатор, который разбивает и преобразует исходные тексты в список токенов.

In [ ]:
model_name = "unitary/toxic-bert"

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Преобразуем текст в номера токенов с помощью токенайзера.

Параметр add_special_tokens = True,значит, что к любому преобразуемому тексту добавляется токен начала (101) и токен конца текста (102).

Параметры truncation=True,max_length=512 означают, что последовательности усекаются до максимальной длины 512 токенов.

In [ ]:
tokenized = data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens = True,truncation = True, max_length = 512))

Посмотрим, как прошла токенизация.

In [ ]:
display(tokenized)

Каждая строка преобразована в список чисел.

Теперь применим метод padding, чтобы после токенизации длины исходных текстов в корпусе были равным (условие для работы BERT). Пусть стандартной длиной вектора n будет длина наибольшего во всём датасете вектора. Остальные векторы дополним нулями.

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [ ]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

Теперь поясним модели, что нули не несут значимой информации. Создадим attention_mask, массив из 0 и 1, который  указывает модели, какие токены следует учитывать (1), а какие игнорировать(0).

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)

Проверим размерности полученных массивов.

In [ ]:
np.array(padded).shape

In [ ]:
attention_mask.shape

Теперь преобразуем тексты в векторы-эмбеддинги, которые содержат данные о соотношении разных слов и их свойствах.

Сделаем цикл по батчам, на каждом шаге преобразуем данные и маску в формат тензоров и передаем их модели unitary/toxic-bert, которая преобразует их в эмбеддинги, и эти эмбеддинги заносятся в список embeddings.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
model.to(device)

In [ ]:
token_ids = torch.tensor(padded).to(device)
attentionMask = torch.tensor(attention_mask ).to(device)

In [ ]:
def get_text_embeddings(data, tokenizer, model, device, max_len=512, batch_size=50):

  # Токенизируем данные
  tokenized = data['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True, max_length=max_len))
  padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
  attention_mask = np.where(padded != 0, 1, 0)

  # Получаем эмбендинги
  text_embeddings = {}
  embeddings=[]

  for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)

    with torch.no_grad():
      model.to(device)
      batch_embeddings = model(batch, attention_mask=attention_mask_batch)

      embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  return embeddings

# получаем векторы
text_embeddings = get_text_embeddings(data, tokenizer, model, device, max_len=512, batch_size=1)

Соберём все эмбеддинги в матрицу признаков features, проверим размерность матрицы и целевого признака и выведем на печать полученные признаки.

In [ ]:
features = np.concatenate(text_embeddings)

print(features.shape[0])
print(data['toxic'].shape[0])


Как видим, преобразование прошло корректно.  Теперь на этих признаках можно обучать модель классификации.

### Подбор модели классификации

Поскольку у нас наблюдается сильный дисбаланс классов в целевом признаке, нужно стратифицировать тренировочную и тестовую выборки, чтобы они сохранили соотношение классов исходных данных. Соотношение тренировочной и тестовой выборки возьмем 1:1.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    data['toxic'],
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE,
    stratify=data['toxic']
)

Проверим размеры выборок.

In [ ]:
print(X_train.shape[0])
print(X_test.shape[0])

Проверим, соблюдены ли пропорции классов в тренировочной и тестовой выборке.

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(12,4))
data['toxic'].value_counts().plot(kind='bar', ax=ax[0], rot=0)
ax[0].set_title("Исходный датасет")
y_train.value_counts().plot(kind='bar', ax=ax[1], rot=0)
ax[1].set_title("Train")
y_test.value_counts().plot(kind='bar', ax=ax[2], rot=0)
ax[2].set_title("Test")
plt.show()

Как видим, пропорции соблюдены верно.

Создадим словарь для моделей, из которых  будем  выбирать лучшую по метрике f1. Для всех моделей, кроме KNeighborsClassifier применим взвешивание классов для решения проблемы дисбаланса классов.

In [ ]:
param_distributions = [
    #словарь для модели KNeighborsClassifier()
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(1, 20),
        'models__weights': ['uniform', 'distance'],

     },
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state = RANDOM_STATE, class_weight = 'balanced')],
        'models__min_samples_leaf': range(1, 10),



    },
    # словарь для модели SVC()
    {
        'models': [SVC(random_state = RANDOM_STATE, probability = True, class_weight = 'balanced')],
        'models__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],


    },
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(random_state = RANDOM_STATE, class_weight = 'balanced', max_iter = 1000)],
        'models__C': [0.5,1,2,3,4,5,6,7,8,9,10],
        'models__solver': ['liblinear', 'saga'],

    },
]

In [ ]:
pipe_final = Pipeline([

    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

Случайный поиск наилучшей модели с кросс-валидацией на 5 выборках по метрике f1:

In [ ]:
randomized_search_all = RandomizedSearchCV(
    pipe_final,
    param_distributions=param_distributions,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    cv=5
)


In [ ]:
randomized_search_all.fit(X_train, y_train)

Выведем таблицу с результатами поиска наилучшей модели.

In [ ]:
pd.set_option('display.max_colwidth', None)
result = pd.DataFrame(randomized_search_all.cv_results_)
display(result[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score'))

Для каждой модели выведем ее лучшие параметры и метрики по убывающей.

In [ ]:
result['param_models']=result['param_models'].astype('str')

Для логистической регрессии:

In [ ]:
result_lr=result[result['param_models'].str.contains('Logistic')]

display(result_lr[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score'))

Для модели к-соседей:

In [ ]:
result_kn=result[result['param_models'].str.contains('KNeighborsClassifier')]

display(result_kn[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score'))

Для SVC и DecisionTreeClassifier подходящие модели не нашлись:

In [ ]:
result_SVC=result[result['param_models'].str.contains('SVC')]

display(result_SVC[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score'))

In [ ]:
result_Decision=result[result['param_models'].str.contains('Decision')]

display(result_Decision[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score'))

Таким образом, наилучшей моделью по метрике f1 признана логистическая регрессия, ее средняя метрика оказалась 0.923.

### Тестирование лучшей модели

Теперь можем рассчитать метрику f1 на тестовой выборке.

In [ ]:
y_pred=randomized_search_all.predict(X_test)

f1_score(y_test,y_pred)

Метрика удовлетворяет условиям задачи.

 Чтобы узнать, не предвзята ли наша модель, сравним её с такой, которая всегда присваивает объектам мажорный класс: DummyClassifier.

In [ ]:
dummy_model = DummyClassifier(strategy = 'constant', constant = 1)

dummy_model.fit(X_train, y_train)

dummy_model_preds = dummy_model.predict(X_test)

dummy_f1 = f1_score(y_test, dummy_model_preds)
print('F1-score =', round(dummy_f1,2))

f1 = 0.22 на тестовой выборке.

Таким образом, наша модель прошла проверку на адекватность.

Для найденной наилучшей модели можно для наглядности вывести матрицу ошибок на тестовой выборке.

In [ ]:
ConfusionMatrixDisplay.from_estimator(randomized_search_all, X_test, y_test)
plt.show()

Как видим, 1 токсичный отзыва модель приняла за нейтральные, а 5 нейтральных несправедливо отнесла к токсичным.

Теоретически, на этапе подбора модели можно было бы изменить порог вероятности, чтобы модель реже относила нейтральные отзывы к токсичным, тогда и ошибку FN можно было бы уменьшить. Но тогда нужно было бы выделять валидационную выборку и на ней подбирать порог, а у нас и так оказалось в наличии немного данных: на 800 строках программа работает около часа.


#  TFIDF

Теперь для интереса попробуем решить задачу без нейронной сети BERT.
У нас есть задача классификации, где целевой признак равен «1» для токсичного текста и «0» для отрицательного, а входные признаки — это слова из корпуса и их величины TF-IDF для каждого текста.

In [ ]:
#достаем обратно исходный датафрейм
data=data_copy.copy(deep=True)

In [ ]:
data.info()

Если для BERT не требовалось лемматизировать текст и очищать его от стоп-слов, то сейчас нам уже придется сделать это.

Функции для очищения от ненужных симловов, лемматизации и удаления  стоп-слов:

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemm_list=lemmatizer.lemmatize(text)
    lemm_text = "".join(lemm_list)

    return lemm_text

def clear_text(text):
    t=re.sub(r'[^a-zA-Z]', ' ', text)
    t=t.split()
    return ' '.join(t)

def remove_stopwords(text):
    text=text.split(' ')
    processed_word_list = []
    for word in text:
        word = word.lower()
        if word not in stopwords:
            processed_word_list.append(word)
    return ' '.join(processed_word_list)

Применяем эти функции к корпусу текстов:

In [ ]:
a=[]

corpus = list(data['text'])

for i in corpus:
         a.append(remove_stopwords(lemmatize(clear_text(i))))

data['text']=a

In [ ]:
display(data.head())

Создаем тренировочную и тестовые выборки, стратифицируя их по признаку 'toxic'.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data['text'],
    data['toxic'],
    test_size = TEST_SIZE,
    random_state = RANDOM_STATE,
    stratify=data['toxic']
)

Будем подбирать гиперпараметры для логистической регрессии, параметры заданы в словаре:

In [ ]:
params={
          'model__C':[.01,.05,.1,.5,1,5,10],
          'model__penalty':['l1','l2']}

Итоговый пайплайн, который состоит из

*   CountVectorizer - преобразователя текста в матрицу числовых данных,
*   TfidfTransformer - преобразователя матрицы в матрицу значений TF-IDF
*   модели логистической регрессии.



In [ ]:
pipeline = Pipeline(
    [
    ('vect', CountVectorizer(min_df=40,ngram_range=(1,4))),
    ('tfidf', TfidfTransformer()),
    ('model',LogisticRegression())
    ])

Подбор гиперпараметров для логистической регрессии с помощью поиска по сетке GridSearchCV.

In [ ]:
grid = GridSearchCV(
    pipeline,
    cv=5,
    n_jobs=-1,
    param_grid=params,
    scoring='f1')

grid.fit(X_train, y_train)

In [ ]:
print('Метрика f1 на тестовой выборке ', grid.score(X_test, y_test))

Метрика на тестовой выборке оказалась приемлемой.

Выведем лучшие модели и их параметры.

In [ ]:
pd.set_option('display.max_colwidth', None)
result = pd.DataFrame(grid.cv_results_)
display(result[
    ['rank_test_score', 'params', 'mean_test_score','params']
].sort_values('rank_test_score'))

Можно произвести отбор  признаков — найти те, которые вносят наибольший вклад в корректное предсказание.

Поиск наиболее важных признаков.

In [ ]:
best_model = grid.best_estimator_
logistic_regression_model = best_model.named_steps['model']
coefficients = logistic_regression_model.coef_

In [ ]:
vectorizer = best_model.named_steps['vect']
feature_names = vectorizer.get_feature_names_out()

In [ ]:
if len(coefficients) == 1:
    coef_df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coefficients[0]
    })

In [ ]:
coef_df['abs_coefficient'] = coef_df['coefficient'].abs()
coef_df = coef_df.sort_values(by='abs_coefficient', ascending=False)

In [ ]:
top_n = 20  # количество признаков
top_features = coef_df.head(top_n)
print(top_features)

In [ ]:
plt.figure(figsize = (10, 8))

plt.barh(top_features['feature'], top_features['coefficient'])

plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('The most important features')
plt.show()

Эти слова внесли наибольший вклад в прогноз, сделанного моделью gs.

# Заключение

Целью задачи было построить модель, которая относит отзывы пользователей к токсичным или нейтральным.

Решение задачи проводилось двумя способами: с помощью нейронной сети BERT и без нее, используя величины TF-IDF в качестве входных признаков.

В первом способе сначала текстовые данные преобразовывались в числовые векторы-эмбеддинги с помощью предобученной модели "unitary/toxic-bert",
а затем на основе полученных числовых признаков подбиралась оптимальная модель для задачи классификации.

Важной особенностью задачи был сильный дисбаланс классов: токсичных отзывов в исходной выборке была только десятая часть.

Оптимальная модель подобрана с помощью случайного поиска, на этапе тестирования наилучшая модель выдала метрику f1 0.94 - больше требуемой.

Во втором способе текст был очищен от ненужных симловов и стоп-слов и лемматизирован. Итоговый пайплайн включал в себя преобразование текста в матрицу числовых данных (CountVectorizer), преобразование в матрицу значений TF-IDF (TfidfTransformer) и модель логистической регресии.

С помощью GridSearchCV был произведен подбор гиперпараметров логистической регрессии. Наилучшая найденная модель выдала метрику f1 на тесте 0.76 - гораздо меньше, чем BERT, но формально она удовлетворяет требованиям.

Для найденной модели выведены слова, которые внесли наибольший вклад при классификации отзывов на нейтральные и токсичные.

Таким образом, задача классификации отзывов решена двумя способами, BERT показала отличный результат, хотя для ее работы исходную выборку пришлось сократить до 800 строк, для которых подсчет эмбеддингов занял 25 минут.
Этот аспект тоже нужно учитывать при выборе модели обработки текстов.